# Transportation Problem in Pulp
This examples shows how a transportation problem can be formulated and solved by PULP.
The mathemetical model for a transportation problem is:
$$ min \sum_{i \in M} \sum_{i \in N} c_{ij} x_{ij}$$
Subject to:
$$ \sum_{j \in W} x_{ij} = a_i \quad \forall \quad i \in D$$
$$ \sum_{i \in D} x_{ij} = b_j \quad \forall \quad j \in W$$
$$x_{ij} \geq 0 \quad \forall \quad i \in D, j \in W $$

With

W : Set of warehouses

D : Set of depots



First import the necessary libraries. Pulp is used to formulate and solve LP problems, numpy is used for matrix operations and itertools.product is used to iterate over a combination of sets

In [2]:
import pulp
import numpy as np
from itertools import product


## Create model and define parameters
First create the model as an LP minimize model

In [4]:
model = pulp.LpProblem('transportation_problem', pulp.LpMinimize)

Create the sets D (depots) and W (Warehouses)

In [5]:
amount_depots = 3
amount_warehouses = 2
set_D = range(amount_depots)
set_W = range(amount_warehouses)


In [7]:
type(set_D)

range

Create the variable x for all combinations of sets D and W. The function product() uses collections (sets, ranges etc) as input and outputs all combinations. This can be used to program 'forall' and 'sum' functions. The category is integer. Other options are Binary and Continuous (Default)

In [8]:
x  = pulp.LpVariable.dicts('x',((i,j) for i,j in product(set_D,set_W)),cat='Integer')

In [9]:
x

{(0, 0): x_(0,_0),
 (0, 1): x_(0,_1),
 (1, 0): x_(1,_0),
 (1, 1): x_(1,_1),
 (2, 0): x_(2,_0),
 (2, 1): x_(2,_1)}

Create the cost matrix, capacities and demands

In [6]:
a = [5,6,5] #Capacities of depots
b = [7,9] # Demands of warehouses
c = np.matrix(([1,2],[4,5],[6,3])) #Cost matrix
print('Cost Matrix:')
print(c)


Cost Matrix:
[[1 2]
 [4 5]
 [6 3]]


## Add cost function and constraints
The first equation added to the model is the cost equation. Equations added after that are seen as constraints.
The cost function is:
$$ min \sum_{i \in M} \sum_{i \in N} c_{ij} x_{ij}$$
The double sum is created by using the function pulp.lpSum to define that a sum has to be taken and itertools.product to get all combinations.

In [7]:
model += pulp.lpSum(c[i,j]*x[i,j] for i,j in product(set_D,set_W))

Total depots capacity constraint:
$$ \sum_{j \in W} x_{ij} = a_i \quad \forall \quad i \in D$$

Note that forall's are done by a loop and sums are done by pulp.lpSum

In [8]:
for i in set_D:
    model += pulp.lpSum(x[i,j] for j in set_W) == a[i]
    
    

Total warehouse demand constraint:
$$ \sum_{i \in D} x_{ij} = b_j \quad \forall \quad j \in W$$

In [9]:
for j in set_W:
    model += pulp.lpSum(x[i,j] for i in set_D) == b[j]

Larger than 0 constraint:
$$x_{ij} \geq 0 \quad \forall \quad i \in M, j \in N $$


In [10]:
for i,j in product(set_D,set_W):
    model += x[i,j] >= 0

## Solve and view result
Now solve the model. Pulp is able to connect to multiple solvers. Here the CPLEX python api is used if available. If not it uses the standard pulp solver

In [11]:
if(pulp.solvers.CPLEX_PY().available()):
    model.solve(pulp.solvers.CPLEX_PY())
    #The api can be called directly by adressing solverModel. This makes it possible to include solve specific functions.
    #For example, the deterministic time of cplex can be accessed by:
    detTime = model.solverModel.get_dettime()
    print('Solved by CPlex with a deterministic time (ticks) of %s' % detTime)
else:
    model.solve()
    print('Solved by PULPs standard solver. Free, but slower than most commercial solvers')

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Found incumbent of value 48.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 8 rows and 3 columns.
Aggregator did 3 substitutions.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.01 sec. (0.02 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.01 sec. (0.02 ticks)
Cplex status= 101
Solved by CPlex with a deterministic time (ticks) of 0.01743030548095703


View the solution:

In [12]:
if(model.status == 1):
    objective = model.objective
    print('Lowest cost found: %s' % pulp.value(objective))
    
    #Put result into numpy matrix
    x_result = np.zeros((amount_depots,amount_warehouses))
    for i,j in product(set_D,set_W):
        x_result[i,j] = x[i,j].varValue
        
    print(x_result)


Lowest cost found: 48.0
[[ 5.  0.]
 [ 2.  4.]
 [ 0.  5.]]
